# Drone Trajectory Planner

In this project, we will develop the drone trajectory planner. This notebook serves as the main file for the project, where we will refer to the instructions and demonstrate our code.

In [1]:
# Import all the files and libraries required for the project

%load_ext autoreload
%autoreload 2
import copy
    
import numpy as np

from src.camera_utils import compute_image_footprint_on_surface, compute_ground_sampling_distance, project_world_point_to_image
from src.data_model import Camera, DatasetSpec
from src.plan_computation import compute_distance_between_images, compute_speed_during_photo_capture, generate_photo_plan_on_grid, compute_plan_time
from src.visualization import plot_photo_plan

import plotly.io as pio
pio.renderers.default = 'iframe'

# Data Specification Model

Now, we will model the dataset specification using the following attributes:

- Overlap: the ratio (in 0 to 1) of scene shared between two consecutive images.
- Sidelap: the ratio (in 0 to 1) of scene shared between two images in adjacent rows.
- Height: the height of the scan above the ground (in meters).
- Scan_dimension_x: the horizontal size of the rectangle to be scanned (in meters).
- Scan_dimension_y: the vertical size of the rectangle to be scanned (in meters).
- exposure_time_ms: the exposure time for each image (in milliseconds).

In this experiment, we will use a camera with the following specifications:
- Overlap: 0.7
- Sidelap: 0.7
- Height: 30.48 meters (100 feet)
- Scan_dimension_x: 150 meters
- Scan_dimension_y: 150 meters
- exposure_time_ms: 2 milliseconds (1/500 seconds)



In [2]:
# Model the nomimal dataset spec

overlap = 0.7
sidelap = 0.7
height = 30.48 # 100 ft
scan_dimension_x = 150
scan_dimension_y = 150
exposure_time_ms = 2 # 1/500 exposure time

dataset_spec = DatasetSpec(overlap, sidelap, height, scan_dimension_x, scan_dimension_y, exposure_time_ms)

print(f"Nominal specs: {dataset_spec}")

Nominal specs: DatasetSpec(overlap=0.7, sidelap=0.7, height=30.48m, scan_area=150x150m, exposure=2ms)


## Camera Model

We want to model the following camera parameters:
- focal length along x axis (in pixels)
- focal length along y axis (in pixels)
- optical center of the image along the x axis (in pixels)
- optical center of the image along the y axis (in pixels)
- Size of the sensor along the x axis (in mm)
- Size of the sensor along the y axis (in mm)
- Number of pixels in the image along the x axis
- Number of pixels in the image along the y axis

In this experiment, we will use a **Skydio VT300L - Wide camera** with the following specifications:
- Focal length along x axis: 4938.56 pixels
- Focal length along y axis: 4936.49 pixels
- Optical center of the image along the x axis: 4095.5 pixels
- Optical center of the image along the y axis: 3071.5 pixels
- Size of the sensor along the x axis: 13.107 mm
- Size of the sensor along the y axis: 9.830 mm
- Number of pixels in the image along the x axis: 8192 pixels
- Number of pixels in the image along the y axis: 6144 pixels

In [3]:
# Define the parameters for Skydio VT300L - Wide camera
# Ref: https://support.skydio.com/hc/en-us/articles/20866347470491-Skydio-X10-camera-and-metadata-overview
fx = 4938.56
fy = 4936.49
cx = 4095.5
cy = 3071.5
sensor_size_x_mm = 13.107
sensor_size_y_mm = 9.830
image_size_x = 8192
image_size_y = 6144

camera_x10 = Camera(fx, fy, cx, cy, sensor_size_x_mm, sensor_size_y_mm, image_size_x, image_size_y)

print(f"X10 camera model: {camera_x10}")

X10 camera model: Camera(fx=4938.56, fy=4936.49, cx=4095.5, cy=3071.5, sensor=13.107x9.830mm, resolution=8192x6144px)


# Camera Operations

Up to now, we have modeled the dataset specification and camera parameters. Next, we will implement some utility functions to perform camera operations. These operations include:
- Project a 3D world point to an image
- Compute image footprint on a surface
- Compute the Ground Sampling Distance

## Camera Operation 1: Project 3D world points into the image


![Camera Projection](assets/image_projection.png)
Reference: [Robert Collins CSE483](https://www.cse.psu.edu/~rtc12/CSE486/lecture12.pdf)


| Formula | Description |
|---------|------------|
| $ x = f_x \frac{X}{Z} $ | Convert 3D world point `X` to 2D image point `x` |
| $ y = f_y \frac{Y}{Z} $ | Convert 3D world point `Y` to 2D image point `y` |
| $ u = x + c_x $ | Convert 2D image point `x` to pixel coordinate `u` |
| $ v = y + c_y $ | Convert 2D image point `y` to pixel coordinate `v` |


Camera operation `project_world_point_to_image` in `src/camera_utils.py` performs the following operations:
1. Takes a 3D world point `[X, Y, Z]` and a camera model as input.
2. Projects the 3D world point to 2D image coordinates `[x, y]` using the pinhole camera model.
3. Converts the 2D image coordinates `[x, y]` to pixel coordinates `[u, v]` using the camera's optical center.

In [4]:
# Define a 3D world point
point_3d = np.array([25, -30, 50], dtype=np.float32)

# Expected pixel coordinates after projection
expected_uv = np.array([6564.80, 109.60], dtype=np.float32)

# Project the 3D world point to image coordinates
uv = project_world_point_to_image(camera_x10, point_3d)

print(f"{point_3d} projected to {uv}")
assert np.allclose(uv, expected_uv, atol=1e-2)

[ 25. -30.  50.] projected to [6564.7803   109.60571]


## Camera Operation 2: Compute Image Footprint on Surface

We have written code to *project* a 3D point into the image. The reverse operation is reprojection, where we take $(x, y)$ and compute the $(X, Y)$ for a given value of $Z$. Note that while going from 3D to 2D, the depth becomes ambiguous so we need the to specify the $Z$.

An image's footprint is the area on the surface which is captured by the image. We can take the two corners of the image and reproject them at a given distance to obtain the width and length of the image.

The camera operation `compute_image_footprint_on_surface` in `src/camera_utils.py` performs the following:
1. Takes a camera model and a distance from the surface (in meters) as input.
2. Get the pixel coordinates of the two corners of the image: (0, 0) and (image_size_x, image_size_y).
3. Reproject the two corners to world coordinates using the existing function `reproject_image_point_to_world`.
4. Calculate the footprint dimensions by finding the absolute difference in the X and Y coordinates of the two reprojected corners.



In [5]:
# Compute image footprint on a surface at 100m distance
footprint_at_100m = compute_image_footprint_on_surface(camera_x10, 100)

# Expected footprint at 100m distance
expected_footprint_at_100m = np.array([165.88, 124.46], dtype=np.float32)

print(f"Footprint at 100m = {footprint_at_100m}")
print(f"Expected footprint at 100m = {expected_footprint_at_100m}")
assert np.allclose(footprint_at_100m, expected_footprint_at_100m, atol=1e-2)


Footprint at 100m = [165.87831271 124.46090238]
Expected footprint at 100m = [165.88 124.46]


In [6]:
# Compute image footprint on a surface at 200m distance
footprint_at_200m = compute_image_footprint_on_surface(camera_x10, 200)

# Expected footprint at 200m distance (should be double the footprint at 100m)
expected_footprint_at_200m = expected_footprint_at_100m * 2

print(f"Footprint at 200m = {footprint_at_200m}")
print(f"Expected footprint at 200m = {expected_footprint_at_200m}")
assert np.allclose(footprint_at_200m, expected_footprint_at_200m, atol=1e-2)

Footprint at 200m = [331.75662541 248.92180476]
Expected footprint at 200m = [331.76 248.92]


## Camera Operation 3: Ground Sampling Distance (GSD)

Ground sampling distance is the length of the ground (in m) captured by a single pixel. This can be calculated from the image footprint (the dimensions of ground captured by the whole sensor) and the number of pixels along the horizontal and vertical dimension.

The camera operation `compute_ground_sampling_distance` in `src/camera_utils.py` uses the following formulas:

$ GSD_x = \frac{footprint_x}{image\_size\_x} $

$ GSD_y = \frac{footprint_y}{image\_size\_y} $

In [7]:
gsd_at_100m = compute_ground_sampling_distance(camera_x10, 100)
expected_gsd_at_100m = 0.0202

print(f"GSD at 100m: {gsd_at_100m}")

assert np.allclose(gsd_at_100m, expected_gsd_at_100m, atol=1e-4)

GSD at 100m: 0.020248817469059807


## Reprojection from 2D to 3D

Reprojection is the reverse of projection. We take a 2D image point and a depth value (Z) and compute the corresponding 3D world point (X, Y, Z).

The camera operation `reproject_image_point_to_world` in `src/camera_utils.py` performs the following:
1. Takes a 2D image point `[u, v]`, a camera model, and a distance from the surface (depth) as input.
2. Converts the pixel coordinates `[u, v]` to normalized image coordinates `[x, y]` using the camera's optical center.
3. Reprojects the normalized image coordinates `[x, y]` to world coordinates `[X, Y, Z]` using the pinhole camera model and the given depth.

Note: There is an ambiguity in depth when going from 2D to 3D we will address.

In [8]:
from src.camera_utils import reproject_image_point_to_world

original_3d_point = np.array([25, -30, 50], dtype=np.float32)
print(f"Original 3D point: {original_3d_point}")

projected_2d = project_world_point_to_image(camera_x10, original_3d_point)
print(f"Projected to 2D: {projected_2d}")

reprojected_3d = reproject_image_point_to_world(camera_x10, projected_2d, original_3d_point[2])
print(f"Reprojected back to 3D: {reprojected_3d}")

print(f"\nDifference between original and reprojected: {np.abs(original_3d_point - reprojected_3d)}")

Original 3D point: [ 25. -30.  50.]
Projected to 2D: [6564.7803   109.60571]
Reprojected back to 3D: [ 25.000004 -30.000002  50.      ]

Difference between original and reprojected: [3.8146973e-06 1.9073486e-06 0.0000000e+00]


### Understanding the Ambiguity in 2D to 3D Reprojection

To go from 2D back to 3D, we **need additional information** - specifically the depth (Z coordinate). This is because:

1. **The projection from 3D to 2D loses depth information** - multiple 3D points along the same ray from the camera center will project to the same 2D pixel.

2. **Without depth, the problem is under-constrained** - given just a 2D pixel coordinate (u, v), there are infinite possible 3D points that could have projected to that pixel.

3. **The depth provides the missing constraint** - once we know the Z distance, we can uniquely determine the X and Y coordinates.

Let's demonstrate this by showing how different depths give different 3D points for the same 2D pixel:

In [9]:
image_point = np.array([4000, 3000])
print(f"Image point: {image_point}")

for depth in [50, 100, 200, 500]:
    world_point = reproject_image_point_to_world(camera_x10, image_point, depth)
    print(f"At depth {depth}m: X={world_point[0]:.2f}, Y={world_point[1]:.2f}, Z={world_point[2]:.2f}")
    reprojected_2d = project_world_point_to_image(camera_x10, world_point)
    print(f"  Projects back to: [{reprojected_2d[0]:.1f}, {reprojected_2d[1]:.1f}]")

Image point: [4000 3000]
At depth 50m: X=-0.97, Y=-0.72, Z=50.00
  Projects back to: [4000.0, 3000.0]
At depth 100m: X=-1.93, Y=-1.45, Z=100.00
  Projects back to: [4000.0, 3000.0]
At depth 200m: X=-3.87, Y=-2.90, Z=200.00
  Projects back to: [4000.0, 3000.0]
At depth 500m: X=-9.67, Y=-7.24, Z=500.00
  Projects back to: [4000.0, 3000.0]


# Compute the Distance Between Photos

The overlap and sidelap are the ratio of the dimensions shared between two photos. We already know the footprint of a single image at a given distance. Can we convert the ratio into actual distances? And how does the distance on the surface relate to distance travelled by the camera?

The function `compute_distance_between_images` in `src/plan_computation.py` performs the following:
1. Take a camera model and dataset specification as input.
2. Validate that the overlap and sidelap are in the range [0, 1).
3. Compute the footprint of a single image on the surface at the specified height using the existing function `compute_image_footprint_on_surface`.
4. Calculate the distance between image centers in the horizontal and vertical directions using the formulas:
   - Distance_x = footprint_x * (1 - overlap)
   - Distance_y = footprint_y * (1 - sidelap)

In [10]:
# Computed distance between images
computed_distances = compute_distance_between_images(camera_x10, dataset_spec)

# Expected distances between images for the nominal dataset spec
expected_distances = np.array([15.17, 11.38], dtype=np.float32)

print(f"Computed distance for X10 camera with nominal dataset specs: {computed_distances}")
assert np.allclose(computed_distances, expected_distances, atol=1e-2)

Computed distance for X10 camera with nominal dataset specs: [15.16791291 11.38070491]


## Experimenting with Photo Spacing

Now, we are going to configure the following parameters and assess how these affect the computed distances:
- Data specification height
- Camera focal length
- Image size

Adjusting flight height, camera focal length, and image resolution directly impacts photo spacing. Pilots can use these relationships to optimize flight plans for coverage, resolution, and efficiency.

### Baseline Case

- **Parameters:**  
    - Camera: `camera_x10`  
    - Dataset Spec: `dataset_spec` (height=30.48m, overlap=0.7, sidelap=0.7)
- **Result:**  
    - Baseline distances:  
        - dx = 15.17 m  
        - dy = 11.38 m  

The spacing is determined by the camera's ground footprint and the required overlap/sidelap. Larger footprints or lower overlap mean greater spacing.

### Doubling Flight Height

- **Change:**  
    - Height increased from 30.48m to 60.96m.
- **Result:** Distances roughly double
    - dx ≈ 30.34 m  
    - dy ≈ 22.76 m  
    - Ratio to baseline ≈ 2x

Increasing flight height increases the ground area captured by each photo (footprint scales linearly with height). Thus, the drone can space photos further apart. Pilots can cover larger areas faster at higher altitudes, but ground resolution decreases.

### Halving Focal Length

- **Change:**  
    - Focal length (fx, fy) reduced by half.
- **Result:** Distances roughly double
    - dx ≈ 30.34 m  
    - dy ≈ 22.76 m  
    - Ratio to baseline ≈ 2x

A shorter focal length makes the camera "wider," increasing the ground footprint. Photos can be spaced further apart. Wider lenses allow faster coverage but reduce image detail.

### Halving Image Size

- **Change:** Image size (pixels) halved in both dimensions.
- **Result:** Distances roughly halve
    - dx ≈ 7.58 m  
    - dy ≈ 5.69 m  
    - Ratio to baseline ≈ 0.5x

Lower image size means each photo covers less ground, so photos must be taken closer together to maintain coverage. Lowering size saves storage but requires denser flight patterns and more photos.

### Summary

| Change                | dx (m)   | dy (m)   | Ratio to Baseline | Analysis                                                   |
|-----------------------|----------|----------|-------------------|------------------------------------------------------------|
| Baseline              | 15.17    | 11.38    | 1x                | Determined by footprint and overlap/sidelap                |
| Double Height         | 30.34    | 22.76    | ~2x               | Higher altitude → larger footprint → greater spacing       |
| Halve Focal Length    | 30.34    | 22.76    | ~2x               | Wider lens → larger footprint → greater spacing            |
| Halve Image Size      | 7.58     | 5.69     | ~0.5x             | Lower resolution → smaller footprint → closer spacing      |


In [11]:
print("Baseline dataset_spec:", dataset_spec)
baseline_distances = compute_distance_between_images(camera_x10, dataset_spec)
print("\nBaseline distances (m) [dx, dy]:", baseline_distances)

# Double height
spec_height = copy.copy(dataset_spec)
spec_height.height = dataset_spec.height * 2.0
dist_height = compute_distance_between_images(camera_x10, spec_height)
print("\nDouble height")
print(" distances:", dist_height, " ratio to baseline:", dist_height / baseline_distances)

# Halve focal length
cam_half_fx = copy.copy(camera_x10)
cam_half_fx.fx *= 0.5
cam_half_fx.fy *= 0.5
dist_half_fx = compute_distance_between_images(cam_half_fx, dataset_spec)
print("\nHalve focal length (fx,fy * 0.5)")
print(" distances:", dist_half_fx, " ratio to baseline:", dist_half_fx / baseline_distances)

# Halve image size
cam_half_res = copy.copy(camera_x10)
cam_half_res.image_size_x //= 2
cam_half_res.image_size_y //= 2
dist_half_res = compute_distance_between_images(cam_half_res, dataset_spec)
print("\nHalf image size (pixels)")
print(" distances:", dist_half_res, " ratio to baseline:", dist_half_res / baseline_distances)

Baseline dataset_spec: DatasetSpec(overlap=0.7, sidelap=0.7, height=30.48m, scan_area=150x150m, exposure=2ms)

Baseline distances (m) [dx, dy]: [15.16791291 11.38070491]

Double height
 distances: [30.33582583 22.76140983]  ratio to baseline: [2. 2.]

Halve focal length (fx,fy * 0.5)
 distances: [30.33582583 22.76140983]  ratio to baseline: [2. 2.]

Half image size (pixels)
 distances: [7.58395646 5.69035246]  ratio to baseline: [0.5 0.5]


## Non-Nadir photos

We have solved for the distance assuming that the camera is facing straight down to the ground. This is called [Nadir scanning](https://support.esri.com/en-us/gis-dictionary/nadir). However, in practice we might want a custom gimbal angle.

We will introduce a double `camera_angle` parameter (which is the angle from the X-axis) in the dataset specification and work out how to adapt the computation.

![Non Nadir Footprint](assets/non_nadir_gimbal_angle.png)

In [12]:
from src.plan_computation import compute_non_nadir_footprint, compute_distance_between_images_non_nadir

# Define a camera angle (in degrees) for non-nadir computation
camera_angle_deg = 30
camera_angle_rad = np.deg2rad(camera_angle_deg)

# Compute for 30 degree tilt
footprint_non_nadir = compute_non_nadir_footprint(camera_x10, dataset_spec.height, camera_angle_rad)
print(f"Footprint at {dataset_spec.height}m with {camera_angle_deg}° tilt: {footprint_non_nadir}")

# Compute the distance between images using the new footprint
distances_non_nadir = compute_distance_between_images_non_nadir(camera_x10, dataset_spec, camera_angle_rad)
print(f"Distance between images (non-nadir, {camera_angle_deg}° tilt): {distances_non_nadir}")

Footprint at 30.48m with 30° tilt: [50.55970971 43.80435364]
Distance between images (non-nadir, 30° tilt): [15.16791291 13.14130609]


# Compute the Maximum Speed For Blur Free Photos

To restrict motion blur due to camera movement to tolerable limits, we need to restrict the speed such that the image contents move less than 1px away. 

The ground sampling distance (GSD) tells us how much ground is covered by a single pixel. From the previous week, we know that this is the maximum movement the camera can have (distance). The speed is then distance divided by time, which is provided by the data model.

The function `compute_speed_during_photo_capture` in `src/plan_computation.py` performs the following operations:
1. Takes a camera model, dataset specification, and allowed movement in pixels as input.
2. Computes the ground sampling distance (GSD) at the specified flight height using the existing function `compute_ground_sampling_distance`.
3. Calculates the maximum allowed ground movement during the exposure time as `allowed_movement_px * GSD`.
4. Converts the exposure time from milliseconds to seconds.
5. Computes the maximum speed as `max_movement / exposure_time_s`.



In [13]:
# Compute the maximum speed during photo capture to avoid motion blur
computed_speed = compute_speed_during_photo_capture(camera_x10, dataset_spec, allowed_movement_px=1)

# Expected speed based on manual calculation
expected_speed = 3.09

print(f"Computed speed during photo captures: {computed_speed:.2f}")
print(f"Expected speed: {expected_speed:.2f}")

assert np.allclose(computed_speed, expected_speed, atol=1e-2)

Computed speed during photo captures: 3.09
Expected speed: 3.09


## Experimenting with Speed Calculation

We are going to configure the following parameters and assess how these affect the computed speed:
- Data specification height
- Camera focal length
- Image size

Adjusting flight height, camera focal length, and image resolution directly impacts the maximum allowable speed for blur-free photos. Pilots can use these relationships to optimize flight plans for speed and image quality.

### Baseline Case

- **Parameters:**  
    - Camera: `camera_x10`  
    - Dataset Spec: `dataset_spec` (height=30.48m, overlap=0.7, sidelap=0.7)
- **Result:**  
    - Computed Speed: 3.09 m/s  

### Doubling Flight Height

- **Change:**  
    - Height increased from 30.48m to 60.96m.
- **Result:** Speed roughly doubles
    - speed ≈ 6.17 m/s

Increasing flight height increases the Ground Sampling Distance (GSD). Since the maximum allowable speed is limited by how much the scene can move during exposure (typically 1 pixel), a larger GSD means the drone can move faster while staying within the same pixel movement limit. Pilots can fly faster at higher altitudes while maintaining sharp images, but this comes at the cost of reduced ground resolution.

### Halving Focal Length

- **Change:**  
    - Focal length (fx, fy) reduced by half.
- **Result:** Speed roughly doubles
    - speed ≈ 6.27 m/s

A shorter focal length makes the camera "wider," increasing the Ground Sampling Distance (GSD). Same as the previous experiment, a larger GSD means the drone can move faster while staying within the same pixel movement limit. Pilots can fly faster with wider lenses while maintaining sharp images, though this comes at the cost of reduced image detail per pixel.

### Halving Image Size

- **Change:** Image size (pixels) halved in both dimensions.
- **Result:** Speed stays the same
    - speed ≈ 3.09 m/s

Lower image size (fewer pixels) doesn't change the Ground Sampling Distance (GSD) because GSD depends on the physical sensor size and focal length, not the number of pixels. Since the maximum allowable speed is determined by GSD and exposure time, halving the pixel count doesn't affect the blur-free speed limit. However, lower resolution means less detail captured per image, requiring pilots to potentially fly lower or use longer exposures for the same level of detail.

### Summary


| Change                | Speed (m/s) | Ratio to Baseline | Analysis                                                   |
|-----------------------|-------------|-------------------|------------------------------------------------------------|
| Baseline              | 3.09        | 1x                | Determined by GSD and exposure time                        |
| Double Height         | 6.17        | ~2x               | Higher altitude → larger GSD → higher allowable speed      |
| Halve Focal Length    | 6.17        | ~2x               | Wider lens → larger GSD → higher allowable speed           |
| Halve Image Size      | 3.09        | ~1x               | Pixel count doesn't affect GSD → no speed change           |

In [14]:
camera_ = copy.copy(camera_x10)
dataset_spec_ = copy.copy(dataset_spec)

computed_speed_ = compute_speed_during_photo_capture(camera_, dataset_spec_)
print(f"Computed distance: {computed_speed_:.2f}")

# Double height
spec_height = copy.copy(dataset_spec)
spec_height.height = dataset_spec.height * 2.0
speed_height = compute_speed_during_photo_capture(camera_x10, spec_height)
print("\nDouble height")
print(" speed:", speed_height, " ratio to baseline:", speed_height / computed_speed_)

# Halve focal length
cam_half_fx = copy.copy(camera_x10)
cam_half_fx.fx *= 0.5
cam_half_fx.fy *= 0.5
speed_half_fx = compute_speed_during_photo_capture(cam_half_fx, dataset_spec)
print("\nHalve focal length (fx,fy * 0.5)")
print(" speed:", speed_half_fx, " ratio to baseline:", speed_half_fx / computed_speed_)

# Halve image size
cam_half_res = copy.copy(camera_x10)
cam_half_res.image_size_x //= 2
cam_half_res.image_size_y //= 2
speed_half_res = compute_speed_during_photo_capture(cam_half_res, dataset_spec)
print("\nHalf image size (pixels)")
print(" speed:", speed_half_res, " ratio to baseline:", speed_half_res / computed_speed_)

Computed distance: 3.09

Double height
 speed: 6.171839564569429  ratio to baseline: 2.0

Halve focal length (fx,fy * 0.5)
 speed: 6.171839564569429  ratio to baseline: 2.0

Half image size (pixels)
 speed: 3.0859197822847144  ratio to baseline: 1.0


# Week 6: Generate Full Flight Plans  

We now have all the tools to generate the full flight plan.

Steps for this week:
1. Define the `Waypoint` data model. What attributes should the data model have?
   1. For Nadir scans, just the position of the camera is enough as we will always look drown to the ground.
   2. For general case (bonus), we also need to define where the drone will look at.
3. Implement the function `generate_photo_plan_on_grid` to generate the full plan.
   1. Compute the maximum distance between two images, horizontally and vertically.
   2. Layer the images such that we cover the whole scan area. Note that you need to take care when the scan dimension is not a multiple of distance between images. Example: to cover 45m length with 10m between images, we would need 4.5 images. Not possible. 4 images would not satisfy the overlap, so we should go with 5. How should we arrange 5 images in the given 45m.
   3. Assign the speed to each waypoint.

$\color{red}{\text{TODO: }}$ Implement:
- `Waypoint` in `src/data_model.py`
- `generate_photo_plan_on_grid` in `src/plan_computation.py`.

In [15]:
computed_plan = generate_photo_plan_on_grid(camera_x10, dataset_spec) 

print(f"Computed plan with {len(computed_plan)} waypoints")

Computed plan with 165 waypoints


In [16]:
MAX_NUM_WAYPOINTS_TO_PRINT = 20

for idx, waypoint in enumerate(computed_plan[:MAX_NUM_WAYPOINTS_TO_PRINT]):
    print(f"Idx {idx}: {waypoint}")

Idx 0: Waypoint(pos=(-75.0, -75.0, 30.5), speed=3.09m/s)
Idx 1: Waypoint(pos=(-60.0, -75.0, 30.5), speed=3.09m/s)
Idx 2: Waypoint(pos=(-45.0, -75.0, 30.5), speed=3.09m/s)
Idx 3: Waypoint(pos=(-30.0, -75.0, 30.5), speed=3.09m/s)
Idx 4: Waypoint(pos=(-15.0, -75.0, 30.5), speed=3.09m/s)
Idx 5: Waypoint(pos=(0.0, -75.0, 30.5), speed=3.09m/s)
Idx 6: Waypoint(pos=(15.0, -75.0, 30.5), speed=3.09m/s)
Idx 7: Waypoint(pos=(30.0, -75.0, 30.5), speed=3.09m/s)
Idx 8: Waypoint(pos=(45.0, -75.0, 30.5), speed=3.09m/s)
Idx 9: Waypoint(pos=(60.0, -75.0, 30.5), speed=3.09m/s)
Idx 10: Waypoint(pos=(75.0, -75.0, 30.5), speed=3.09m/s)
Idx 11: Waypoint(pos=(75.0, -64.3, 30.5), speed=3.09m/s)
Idx 12: Waypoint(pos=(60.0, -64.3, 30.5), speed=3.09m/s)
Idx 13: Waypoint(pos=(45.0, -64.3, 30.5), speed=3.09m/s)
Idx 14: Waypoint(pos=(30.0, -64.3, 30.5), speed=3.09m/s)
Idx 15: Waypoint(pos=(15.0, -64.3, 30.5), speed=3.09m/s)
Idx 16: Waypoint(pos=(0.0, -64.3, 30.5), speed=3.09m/s)
Idx 17: Waypoint(pos=(-15.0, -64.3, 30

## Time Computation

**Note**
- **Max drone speed:** 16 m/s  
- **Max acceleration:** 3.5 m/s²  
- **v0:** starting speed (here, v_photo)

**Triangular profile**

Occurs when the segment is too short to reach cruise (max) speed. The drone accelerates from v0 to a peak v_peak (< v_max) and then symmetrically decelerates back to v0.

Key relations (symmetric accel/decel, |a| = a_max):
- Peak speed:
    $v_{\text{peak}}=\sqrt{v_0^2 + a_{\max}\cdot d}$

- Acceleration time:
    $t_{\text{acc}}=\frac{v_{\text{peak}}-v_0}{a_{\max}}$

- Total travel time:
    $t_{\text{total}}=2\,t_{\text{acc}}$

Use this when the available distance d is too short to accelerate to v_max and then decelerate.

**Trapezoidal profile**

Occurs when the segment is long enough to reach v_max. The speed profile: accelerate from v0 → v_max, cruise at v_max, then decelerate back to v0.

Distances and times:
- Distance for accel+decel:
    $d_{\text{acc+dec}}=\frac{v_{\max}^2 - v_0^2}{a_{\max}}$

- Cruise distance:
    $d_{\text{cruise}}=d - d_{\text{acc+dec}}$

- Acceleration time:
    $t_{\text{acc}}=\frac{v_{\max}-v_0}{a_{\max}}$

- Cruise time:
    $t_{\text{cruise}}=\frac{d_{\text{cruise}}}{v_{\max}}$

- Total travel time:
    $t_{\text{total}}=2\,t_{\text{acc}}+t_{\text{cruise}}$

 **Decision rule (simple algorithm)**
1. Compute $v_{\text{peak}}=\sqrt{v_0^2 + a_{\max}\cdot d}$  
2. If $v_{\text{peak}}\le v_{\max}$ → use **triangular** profile (peak = v_peak).  
3. Else → use **trapezoidal** profile (accelerate to v_max, cruise, decelerate).

**Why it matters for the planner**
- Provides the minimum feasible travel time while ensuring the drone can decelerate to v_photo before capture.
- Affects mission duration, energy usage, and safety (braking capability).
- Practical use: compute per-segment profile (triangular vs trapezoidal) using (v0 = v_photo, v_max, a_max) to obtain accurate travel times and ensure safe deceleration to capture speed.

In [17]:
from src.plan_computation import compute_plan_time, compute_speed_during_photo_capture

v_photo = compute_speed_during_photo_capture(camera_x10, dataset_spec, allowed_movement_px=1)
positions = [np.array([wp.x, wp.y, wp.z]) for wp in computed_plan]
exposure_s = dataset_spec.exposure_time_ms / 1000.0

total_time_s, segments = compute_plan_time(positions, v_photo, exposure_time_s=exposure_s)
print(f"Estimated total mission time: {total_time_s:.1f} s ({total_time_s/60:.1f} min)")
print("Number of segments:", len(segments))
for idx, segment in enumerate(segments):
    print(f"Segment {idx}: {segment}")
    

Estimated total mission time: 441.0 s (7.4 min)
Number of segments: 164
Segment 0: {'index': 0, 'distance': 15.0, 'travel_time_s': 2.7368812378527148, 'profile': {'type': 'triangular', 'v_peak': 7.875461948526965, 't_acc': 1.3684406189263574}, 'capture_time_s': 0.002}
Segment 1: {'index': 1, 'distance': 15.0, 'travel_time_s': 2.7368812378527148, 'profile': {'type': 'triangular', 'v_peak': 7.875461948526965, 't_acc': 1.3684406189263574}, 'capture_time_s': 0.002}
Segment 2: {'index': 2, 'distance': 15.0, 'travel_time_s': 2.7368812378527148, 'profile': {'type': 'triangular', 'v_peak': 7.875461948526965, 't_acc': 1.3684406189263574}, 'capture_time_s': 0.002}
Segment 3: {'index': 3, 'distance': 15.0, 'travel_time_s': 2.7368812378527148, 'profile': {'type': 'triangular', 'v_peak': 7.875461948526965, 't_acc': 1.3684406189263574}, 'capture_time_s': 0.002}
Segment 4: {'index': 4, 'distance': 15.0, 'travel_time_s': 2.7368812378527148, 'profile': {'type': 'triangular', 'v_peak': 7.875461948526965

# Week 7: Visualize Flight Plans

This week, we will use a third party plotting framework called [Plotly](https://plotly.com/python/) to visualize our plans. Please follow this [tutorial](https://www.kaggle.com/code/kanncaa1/plotly-tutorial-for-beginners) to gain some basic experience with Plotly, and then come up with your own visualization function. You are free to choose to come up with your own visualization, and use something other than Plotly.

$\color{red}{\text{TODO: }}$ Implement `plot_photo_plan` in `src/visualization.py`

In [ ]:
fig = plot_photo_plan(computed_plan)
fig.show()

$\color{red}{\text{TODO: }}$ Perform the following experiments (and any other you can think of) where we change just one parameter of the input camera/dataset specification and observe the change in the output plan. 

1. Change overlap and confirm it affects the consecutive images
2. Change sidelap and confirm it does not affect the consecutive images
3. Change the height of the scan and document the affect on scan plans
4. Change exposure time

Each experiment should specify: 
1. Input params you are changing
2. Impact you observe
3. explanation behind the change in output (intuition based or a text explanation is preffered over using equations)
4. Practical implication of the correlation: how can I drone pilot use this result

In [ ]:
camera_ = copy.deepcopy(camera_x10)
dataset_spec_ = copy.deepcopy(dataset_spec)

dataset_spec_.exposure_time_ms = 1000

print(camera_, dataset_spec_)

fig = plot_photo_plan(generate_photo_plan_on_grid(camera_, dataset_spec_))
fig.show()